In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device : {device}")

Device : cpu


In [ ]:
class TestNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [ ]:
model = TestNet().to(device)
print(model)

TestNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0])


In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])


In [8]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2648,  0.1463,  0.2462,  0.1117,  0.2997,  0.0306,  0.4290, -0.1961,
          0.4867,  0.3651, -0.3340,  0.0801, -0.3920, -0.3854,  0.3546,  0.7890,
         -0.3652, -0.3499, -0.4118,  0.1029],
        [-0.0891,  0.4811,  0.0716, -0.2966,  0.6760,  0.0665,  0.4799, -0.0574,
          0.1629,  0.4939, -0.4726, -0.1384, -0.2314, -0.3323,  0.1190,  0.2844,
         -0.3771, -0.2106, -0.2698,  0.3197],
        [-0.1695,  0.0671,  0.4719, -0.0555,  0.2252, -0.1205,  0.3093,  0.0520,
          0.2076,  0.6841, -0.1949,  0.1184, -0.1705, -0.1990,  0.2106,  0.5059,
         -0.5691, -0.2464, -0.3637,  0.0313]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.1463, 0.2462, 0.1117, 0.2997, 0.0306, 0.4290, 0.0000, 0.4867,
         0.3651, 0.0000, 0.0801, 0.0000, 0.0000, 0.3546, 0.7890, 0.0000, 0.0000,
         0.0000, 0.1029],
        [0.0000, 0.4811, 0.0716, 0.0000, 0.6760, 0.0665, 0.4799, 0.0000, 0.1629,
         0.4939, 0.0000, 0.0000, 0.0000, 0.0000, 0.11

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[ 0.0358,  0.1801, -0.2550, -0.2508, -0.0892, -0.0405,  0.2534, -0.1120,
          0.1393,  0.3653],
        [ 0.0598,  0.2266, -0.1791, -0.4355, -0.1078,  0.0018,  0.1642, -0.1114,
          0.0937,  0.2685],
        [ 0.1371,  0.2496, -0.1163, -0.3588, -0.0912, -0.0481,  0.3017,  0.0309,
          0.2196,  0.1529]], grad_fn=<AddmmBackward0>)


In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: TestNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0044,  0.0180,  0.0252,  ..., -0.0100,  0.0031, -0.0066],
        [-0.0304, -0.0118,  0.0128,  ..., -0.0043, -0.0268, -0.0080]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0037, -0.0156], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0019, -0.0150, -0.0429,  ...,  0.0129,  0.0205,  0.0380],
        [-0.0167,  0.0107, -0.0199,  ...,  0.0382,  0.0294, -0.0253]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: 